In [ ]:
def readSequencesFromFastaFile(filepath):

    reads = []
    currentSeq = ""

    f = open(filepath, "r")
    for line in f:

        if(line[0] == '>'):
            if(currentSeq != ""):
                reads.append(currentSeq)
                currentSeq = ""
        else:
            currentSeq += line.strip("\r\n")
            
    if(currentSeq != ""):
        reads.append(currentSeq)

    f.close()
    return reads

In [ ]:
def assemble(reads):
    main_seq = reads.pop(0)
    while(len(reads) > 1): 
        
        (cover, r) = (0, -1)
        
        for i in range(1, len(reads)):

            min_length = min(len(main_seq), len(reads[i]))
            
            # préfixe : on compare la début de la séquence principale avec la fin de la séquence reads[i] :
            #               MAINSEQ -->
            #           <-- READI 
            prefix_cover = -min_length + 1
            while(prefix_cover < 0 and reads[i][len(reads[i]) + prefix_cover:] != main_seq[:abs(prefix_cover)]):
                prefix_cover+=1

            if(abs(prefix_cover) > abs(cover)): 
                (cover, r) = (prefix_cover, i)

            # suffixe : on compare la fin de la séquence principale avec le début de la séquence reads[i] :
            #           <-- MAINSEQ
            #                 READI --> 
            sufix_cover = min_length - 1
            while(sufix_cover > 0 and main_seq[len(main_seq) - sufix_cover:] != reads[i][:sufix_cover]):
                sufix_cover-=1

            if(sufix_cover > abs(cover)): 
                (cover, r) = (sufix_cover, i)
            
            # Méthode un peu sale pour sortir de la boucle for lorsqu'on trouve un chevauchement de la taille maximale 
            # (à savoir un chevauchement de la taille de la plus petite chaîne -1) :
            if(abs(cover) == min_length - 1): break

        if(cover != 0):
            r_seq = reads.pop(r)
            if(cover < 0):
                main_seq = r_seq[:len(r_seq) - cover] + main_seq
            else:
                main_seq = main_seq[:len(main_seq) - cover] + r_seq
        else:
            return "Erreur, au moins une séquence ne contient pas de chevauchement (de longueur > 0) dans les chaînes proposées"
    
    return main_seq

In [ ]:
def assembleOpti(reads):
    main_seq = reads[0]
    while(len(reads) > 1): 
        
        max_overlap = 0
        r = -1
        reads2 = [main_seq]
        
        for i in range(1, len(reads)):

            readi = reads[i]
            min_length = min(len(main_seq), len(readi))

            # On compare le début de la séquence principale avec la fin de la séquence reads[i] :
            #              MAINSEQ --> 
            #          <-- READI
            readi_slice = readi[len(readi) - min_length + 1:]
            main_seq_slice = main_seq[:min_length - 1]

            while(len(readi_slice) > 0 and readi_slice != main_seq_slice):
                readi_slice = readi_slice[1:]
                main_seq_slice = main_seq_slice[:-1]
                

            if(len(readi_slice) > abs(max_overlap)):
                max_overlap = -len(readi_slice)
                if(r != -1): 
                    reads2.append(reads[r])
                r = i
            
            if(abs(max_overlap) == min_length - 1):
                reads2.extend(reads[i+1:])
                break

            # On compare la fin de la séquence principale avec le début de la séquence reads[i] :
            #           <-- MAINSEQ
            #                 READI --> 
            readi_slice = readi[:min_length - 1]
            main_seq_slice = main_seq[len(main_seq) - min_length + 1:]

            while(len(readi_slice) > 0 and readi_slice != main_seq_slice):
                readi_slice = readi_slice[:-1]
                main_seq_slice = main_seq_slice[1:]

            if(len(readi_slice) > abs(max_overlap)):
                if(r != i and r!=-1): 
                    reads2.append(reads[r])
                max_overlap = len(readi_slice)
                r = i
            
            if(abs(max_overlap) == min_length - 1):
                reads2.extend(reads[i+1:])
                break

            if(len(readi_slice) < abs(max_overlap) and r != i):
                reads2.append(readi)

        if(max_overlap != 0):
            r_seq = reads[r]
            if(max_overlap < 0):
                main_seq = r_seq[:len(r_seq) + max_overlap] + main_seq
            else:
                main_seq = main_seq + r_seq[max_overlap:]
        else:
            # raise Exception("Erreur, au moins une séquence ne contient pas de chevauchement (de longueur > 0) dans les chaînes proposées")
            
            print("Erreur : au moins une séquence ne contient pas de chevauchement (de longueur > 0) dans les chaînes proposées")
            return main_seq

        reads = reads2
    return main_seq

In [ ]:
# Test :

# Lecture du fichier fasta des reads de E.Coli (small) :

readsFasta = readSequencesFromFastaFile("./fasta/Escherichia_coli_SMALL_READS.fasta")

# print(readsFasta[0])
# Alignement des séquences :
result = assembleOpti(readsFasta)

# Comparaison de la séquence obtenue avec la vraie séquence (donnée dans un des fichiers fasta)  
# Lecture du fichier fasta du résultat d'alignement attendu E.Coli (small) :

fastaResult = readSequencesFromFastaFile("./fasta/Escherichia_coli_SMALL.fasta").pop()

# Comparaison :

print("Résultat obtenu  = " + result)
print("Résultat attendu = " + fastaResult)
print("\n> Correspondance : " + str(result == fastaResult))


In [ ]:
def assembleBruijn(reads, k):
    nodes = {}

    if k < round(len(reads[0])/2) or k> len(reads[0]) - 1: 
        k = len(reads[0]) - 1
    
    # utilisation d'un set pour éviter les redondances de kmers
    kmers = set()
    for r in reads:

        while len(r) >= k:
            kmers.add(r[:k])
            r = r[1:]

    # Add a kmer to the graph 
    # nodes are represented with 2 attributes:
    # - next nodes (name)
    # - previous nodes (name)
    for kmer in kmers:
        key1 = kmer[:k - 1]
        key2 = kmer[1:]

        #First node :
        node1 = nodes.get(key1)
        if not node1:
            node1 = {'next': {}, 'previous': {}}
        
        if not node1['next'].get(key2) :
            node1['next'][key2] = 0
        node1['next'][key2] += 1

        # Add second node :
        node2 = nodes.get(key2)
        if not node2:
            node2 = {'next': {}, 'previous': {}}
                  
        if not node2['previous'].get(key1) :
            node2['previous'][key1] = 0
        node2['previous'][key1] += 1

        nodes[key1] = node1
        nodes[key2] = node2


    # assemble the graph :
    base_key = next(iter(nodes))
    node = nodes[base_key]
    sequence = base_key
    
    # run through nodes in both left and right directions
    while len(node['previous']) > 0:
        key = next(iter(node['previous']))
        node = nodes.pop(key)
        sequence = key[:1] + sequence

    if len(node['previous']) > 1:
        raise Exception("Erreur, au moins une séquence ne contient pas de chevauchement (de longueur > 0) dans les chaînes proposées")

    node = nodes.pop(base_key)
    while len(node['next']) > 0:
        key = next(iter(node['next']))
        node = nodes.pop(key)
        sequence += key[-1:]

    if len(node['next']) > 1:
        raise Exception("Erreur, au moins une séquence ne contient pas de chevauchement (de longueur > 0) dans les chaînes proposées")
    
    if len(nodes) > 0:
        raise Exception("Erreur, au moins une séquence ne contient pas de chevauchement (de longueur > 0) dans les chaînes proposées")

    return sequence
k=100
coroReads = readSequencesFromFastaFile('./fasta/MN908947.3_READS_MIXED.fasta')
start_time = time.time()
assembleBruijn(coroReads, k)
print("--- k = "+str(k)+"   = "+str(time.time() - start_time)+" seconds ---")

In [ ]:
# Comparaison du temps d'exécution en faisant varier k :
# Tests temps de calcul :
import time

coroReads = readSequencesFromFastaFile('./fasta/MN908947.3_READS_MIXED.fasta')
 
# k : 51 -> 100
for k in range(51, 101):
    start_time = time.time()
    assembleBruijn(coroReads, k)
    print("--- k = "+str(k)+"   = "+str(time.time() - start_time)+" seconds ---")

In [ ]:
# Test :

# Lecture du fichier fasta des reads de E.Coli (small) :
readsFasta = readSequencesFromFastaFile("./fasta/Escherichia_coli_SMALL_READS.fasta")
# print(readsFasta[0])
# Alignement des séquences :
result = assembleBruijn(readsFasta, 0)

# Comparaison de la séquence obtenue avec la vraie séquence (donnée dans un des fichiers fasta)  
# Lecture du fichier fasta du résultat d'alignement attendu E.Coli (small) :
fastaResult = readSequencesFromFastaFile("./fasta/Escherichia_coli_SMALL.fasta").pop()
# Comparaison :

print("Résultat obtenu  = " + result)
print("Résultat attendu = " + fastaResult)
print("\n> Correspondance : " + str(result == fastaResult))